# Convert XGboost Model to SQL Code
In some real-life cases, we want to deploy the ML model quickly but this production system environment is very complicated. So we could use this tool that converts this XGBoost model to native SQL code. SQL is very popular in the legacy system. Deploy those native model codes without any ML framework. This tool could help everybody to turn all your Machine Learning model to productionization quickly. Wish this tool could make you happy and cozy ^^ !.

In [1]:
import lightgbm2Java, lightgbm2SQL
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [2]:
from xgboost import XGBClassifier

In [3]:
import xgboost as xgb


# Train Classifier with titanic data

In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
train_df = pd.read_csv('https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
cat_cols = ['Sex','Embarked']
numerical_cols =  ['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']
onehot_cols = []
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_df[cat_cols].to_numpy().reshape([-1,len(cat_cols)]))
enc.categories_
encoded_array = enc.transform(train_df[cat_cols].to_numpy().reshape([-1,len(cat_cols)])).toarray()
feature_dim = encoded_array.shape[1]
encoded_array = encoded_array.reshape([-1,feature_dim,1])

new_cat_cols = []
for row in enc.categories_:
    for col in row:
        new_cat_cols.append(col)
        
for i in range(feature_dim):
    train_df[new_cat_cols[i]] = encoded_array[:,i]
    onehot_cols.append('f'+str(i))
    
for c in cat_cols:
    del train_df[c]
train_df.head(2)

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,female,male,C,Q,S,NaN
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0.0,1.0,0.0,0.0,1.0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1.0,0.0,1.0,0.0,0.0,0.0


In [6]:
new_cat_cols

['female', 'male', 'C', 'Q', 'S', nan]

In [7]:
y = train_df.pop('Survived')
cols = numerical_cols + new_cat_cols
X_train, X_test, y_train, y_test = train_test_split(train_df[cols],
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [8]:
model = XGBClassifier(max_depth=2,n_estimators=2)
model.fit(X_train, y_train)

[20:00:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/kentshih/PycharmProjects/kent-ai-learning-notebook/venv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=2, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# One Hot Encoding2SQL

In [9]:
from Utility import xgb2SQL

In [11]:
print(xgb2SQL.genOneHotSql(cat_cols,enc.categories_,numerical_cols))

CREATE VIEW one_hot_view as 
select `Pclass`,`Age`,`Fare`,`SibSp`,`Parch`,
  case when `Sex`='female' then 1 else 0 end as `female`,
  case when `Sex`='male' then 1 else 0 end as `male`,
  case when `Embarked`='C' then 1 else 0 end as `C`,
  case when `Embarked`='Q' then 1 else 0 end as `Q`,
  case when `Embarked`='S' then 1 else 0 end as `S`,
  case when `Embarked`='nan' then 1 else 0 end as `nan`,
  `id`

from raw_data;


# Boost Tree to SQL

In [13]:
tree = xgb2SQL.doProcess(model.get_booster(), 'one_hot_view')
print(tree)

CREATE VIEW moon_table AS 
	SELECT id,
		CASE
			WHEN ((female < 0.5) OR (female IS NULL))
			AND (Age < 6.5)
		THEN 0.184615403
			WHEN ((female < 0.5) OR (female IS NULL))
			AND ((Age >= 6.5) OR (Age IS NULL))
		THEN -0.402227193
			WHEN (female >= 0.5)
			AND ((Pclass < 3) OR (Pclass IS NULL))
		THEN 0.537313461
			WHEN (female >= 0.5)
			AND (Pclass >= 3)
		THEN -0.0151260514
		END AS column_0, 
		CASE
			WHEN ((female < 0.5) OR (female IS NULL))
			AND ((Pclass < 2) OR (Pclass IS NULL))
		THEN -0.0714709014
			WHEN ((female < 0.5) OR (female IS NULL))
			AND (Pclass >= 2)
		THEN -0.322500855
			WHEN (female >= 0.5)
			AND ((Pclass < 3) OR (Pclass IS NULL))
		THEN 0.412111074
			WHEN (female >= 0.5)
			AND (Pclass >= 3)
		THEN -0.0107414303
		END AS column_1
	FROM one_hot_view;

SELECT id,
    1 / ( 1 + EXP ( - (
    column_0
	+ column_1 ) ) ) AS score
FROM moon_table


# Test SQL

In [14]:
import sqlite3
con = sqlite3.connect(':memory:')
cur = con.cursor()
from math import exp
def sqlite_exp(x):
    return exp(x)
con.create_function("exp", 1, sqlite_exp)
cur.execute("CREATE TABLE raw_data ({});".format(",".join(list(test_df.columns)))) # use your column names here
con.commit()
qmark = ",".join(['?']*test_df.shape[1])
cur.executemany("INSERT INTO raw_data VALUES ({})".format(qmark), test_df.to_numpy())
cur.execute('select * from raw_data limit 2').fetchall()

[(892, 3, 'Kelly, Mr. James', 'male', 34.5, 0, 0, '330911', 7.8292, None, 'Q'),
 (893,
  3,
  'Wilkes, Mrs. James (Ellen Needs)',
  'female',
  47.0,
  1,
  0,
  '363272',
  7.0,
  None,
  'S')]

In [15]:
cur.execute("""

CREATE VIEW one_hot_view as 
select `Pclass`,`Age`,`Fare`,`SibSp`,`Parch`,
  case when `Sex`='female' then 1 else 0 end as `female`,
  case when `Sex`='male' then 1 else 0 end as `male`,
  case when `Embarked`='C' then 1 else 0 end as `C`,
  case when `Embarked`='Q' then 1 else 0 end as `Q`,
  case when `Embarked`='S' then 1 else 0 end as `S`,
  case when `Embarked`='nan' then 1 else 0 end as `nan`,
  `PassengerId` as `id`

from raw_data;




""")
con.commit()


In [16]:
cur.execute("""

CREATE VIEW moon_table AS 
	SELECT id,
		CASE
			WHEN ((female < 0.5) OR (female IS NULL))
			AND (Age < 6.5)
		THEN 0.184615403
			WHEN ((female < 0.5) OR (female IS NULL))
			AND ((Age >= 6.5) OR (Age IS NULL))
		THEN -0.402227193
			WHEN (female >= 0.5)
			AND ((Pclass < 3) OR (Pclass IS NULL))
		THEN 0.537313461
			WHEN (female >= 0.5)
			AND (Pclass >= 3)
		THEN -0.0151260514
		END AS column_0, 
		CASE
			WHEN ((female < 0.5) OR (female IS NULL))
			AND ((Pclass < 2) OR (Pclass IS NULL))
		THEN -0.0714709014
			WHEN ((female < 0.5) OR (female IS NULL))
			AND (Pclass >= 2)
		THEN -0.322500855
			WHEN (female >= 0.5)
			AND ((Pclass < 3) OR (Pclass IS NULL))
		THEN 0.412111074
			WHEN (female >= 0.5)
			AND (Pclass >= 3)
		THEN -0.0107414303
		END AS column_1
	FROM one_hot_view;


""")
con.commit()


In [17]:
cur.execute("""
SELECT id,
    1 / ( 1 + EXP ( - (
    column_0
	+ column_1 ) ) ) AS score
FROM moon_table

""").fetchall()

[(892, 0.32635268544716695),
 (893, 0.49353349014713005),
 (894, 0.32635268544716695),
 (895, 0.32635268544716695),
 (896, 0.49353349014713005),
 (897, 0.32635268544716695),
 (898, 0.49353349014713005),
 (899, 0.32635268544716695),
 (900, 0.49353349014713005),
 (901, 0.32635268544716695),
 (902, 0.32635268544716695),
 (903, 0.3837413285837312),
 (904, 0.7209994326380124),
 (905, 0.32635268544716695),
 (906, 0.7209994326380124),
 (907, 0.7209994326380124),
 (908, 0.32635268544716695),
 (909, 0.32635268544716695),
 (910, 0.49353349014713005),
 (911, 0.49353349014713005),
 (912, 0.3837413285837312),
 (913, 0.32635268544716695),
 (914, 0.7209994326380124),
 (915, 0.3837413285837312),
 (916, 0.7209994326380124),
 (917, 0.32635268544716695),
 (918, 0.7209994326380124),
 (919, 0.32635268544716695),
 (920, 0.3837413285837312),
 (921, 0.32635268544716695),
 (922, 0.32635268544716695),
 (923, 0.32635268544716695),
 (924, 0.49353349014713005),
 (925, 0.49353349014713005),
 (926, 0.383741328583731